In [ ]:
import yfinance as yf 


sp500 = yf.Ticker("^GSPC")

#get the history using query 
sp500 = sp500.history(period="max")

sp500

#remove the collumns we don't neeed 
del sp500["Dividends"]
del sp500["Stock Splits"]


#set up target to predict the directionality weather it will go up or down 
sp500["Tomorow"] = sp500["Close"].shift(-1)
sp500



In [ ]:
# set up a target based on tomorow's price
# The target is what we're going to try to predict with machine learning 


#this return's a boolean so you'll need a to turn into int
sp500["Target"] = (sp500["Tomorow"] > sp500["Close"]).astype(int) 
sp500

In [ ]:

sp500 = sp500.loc["1990-01-01":].copy()
sp500

In [ ]:
#start to train machine learning

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, min_samples_split=100, random_state=1)

#time series data (recomened to not use cross validation)
train = sp500.iloc[:-100]
test = sp500.iloc[-100:]

predictors = ["Close", "Open", "High", "Low", "Volume"]
model.fit(train[predictors], train["Target"])




In [ ]:
#measure acurate metric of machine learning for the model 
from sklearn.metrics import precision_score
# generate predictions using the model 
preds = model.predict(test[predictors])

import pandas as pd 
preds = pd.Series(preds, index=test.index)

#calculate presicion score 
precision_score(test["Target"], preds)

In [ ]:
#plot the predictions
combined = pd.concat([test["Target"], preds], axis=1)
# combined.plot()

In [ ]:
#back test 
def predict(train, test, predictors, model): 

    model.fit(train[predictors], train["Target"])
    preds = model.predict(test[predictors]) #generate model
    preds= pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined 


def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []

    for i in range(start, data.shape[0], step):

        train = data.iloc[0:i].copy()
        test = data.iloc[i: (i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    return pd.concat(all_predictions)


In [ ]:

predictions = backtest(sp500, model, predictors)
predictions["Predictions"].value_counts()

In [ ]:
precision_score(predictions["Target"], predictions["Predictions"])

In [ ]:
horizons = [2, 5, 60, 250, 1000]
new_predictions = []

for horizon in horizons:

    rolloing_averages = sp500.rolling(horizon).mean()
    ratio_collumn = f"Close_Ratio_{horizon}"
    sp500[ratio_collumn] = sp500["Close"] / rolloing_averages["Close"]

    trend_column = f"Trend_{horizon}"
    sp500[trend_column] = sp500.shift(1).rolling(horizon).sum()["Target"]

    new_predictions += [ratio_collumn, trend_column]

In [ ]:
sp500 = sp500.dropna()
sp500

In [ ]:
model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)

def predict(train, test, predictors, model): 

    model.fit(train[predictors], train["Target"])
    preds = model.predict_proba(test[predictors])[:,1] #generate model

    preds[preds >= .6] = 1
    preds[preds < .6] = 0

    preds= pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined 

In [ ]:
predictions = backtest(sp500, model, new_predictions)

predictions["Predictions"].value_counts()

In [ ]:
precision_score(predictions["Target"], predictions["Predictions"])